In [ ]:
import analysis
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, gridplot
from bokeh.palettes import brewer

In [ ]:
output_notebook()

In [ ]:
# Download csv data here: https://research.stlouisfed.org/fred2/downloaddata/
# and change this variable to point to the main the unzipped directory
analysis.FRED_dir    = '/Users/tbk/FRED2_csv_2/'

# Download this file: http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt
# and change this variable to point to the text file
analysis.county_file = '/Users/tbk/national_county.txt'

### First, we will need to get a list of files from the FRED directory, and a dictionary of counties:

In [ ]:
counties   = analysis.get_county_dict(analysis.county_file)
FRED_files = analysis.get_FRED_files(analysis.FRED_dir+'README_SERIES_ID_SORT.txt')

To demonstrate the format of the above:

In [ ]:
print counties['27053'] # the best county in the world!
print FRED_files[0]

### Next, we will extract the files which correspond to PCPI or population data, and aggregate them into a single dataframe:

In [ ]:
# This cell will take several minutes to run!

# Load the PCPI files into a dataframe
PCPI_files = analysis.get_PCPI_files(FRED_files, counties.keys())
PCPI_data  = analysis.aggregate(PCPI_files, 'PCPI', verbose=False, save='')

# Load the POP files into a dataframe
# We will only use counties that have both sets,
# so we will only load data that was accessed in the PCPI dataframe
POP_files  = analysis.get_POP_files(FRED_files, PCPI_data.loaded, counties)
POP_data   = analysis.aggregate(POP_files, 'POP', verbose=False, save='')

# ...and then remove those few sets which had PCPI data but no POP data
to_remove  = [PCPI_FIP+'_'+PCPI_data.label for PCPI_FIP in PCPI_data.loaded if PCPI_FIP not in POP_data.loaded]
PCPI_data.drop(to_remove, axis=1, inplace=True)

### Let's transform the data into something a bit more manageable:

In [ ]:
# Normalize PCPI values to 2015 USD and convert to units of thousands
PCPI_norm  = analysis.deflate(PCPI_data/1e3, norm_filename='C/P/I/CPIAUCSL.csv', base='2015')


# Take logarithm of POP data, then convert from thousands (add 3)
POP_log = POP_data.apply(np.log10) + 3

### Now we can do all kinds of cool things. Let's start with just plotting the data from the first year (1970) and the last (2013):

In [ ]:
year1 = '1970'
year2 = '2013'

f = figure(width = 900, height = 540, x_range=[0, 100], y_range=[0, 8], title='PCPI vs POP', \
           x_axis_label='Per Capita Personal Income (2015 Dollars, Thousands)', y_axis_label='Log County Population')
f.circle(PCPI_norm[year1].values[0], POP_log[year1].values[0], alpha=0.1, color='royalblue', legend=year1)
f.circle(PCPI_norm[year2].values[0], POP_log[year2].values[0], alpha=0.1, color='firebrick', legend=year2)
show(f)

### Let's look at some individual counties over time. First, we'll invert the `counties` dictionary to make it more human-friendly:

In [ ]:
# Double check that the names of counties uniquely identify each position
# (otherwise we'll have errors which won't raise exceptions)
len(set(counties.keys())) == len(set(counties.values()))

In [ ]:
# We're in the clear, so let's invert the dictionary
counties_inv = {name : FIPS for FIPS, name in counties.items()}

# and test it out
print counties_inv['Hennepin County, MN']
print counties[counties_inv['Hennepin County, MN']]

### Now we can easily look up individual counties by name:

In [ ]:
names = [
    'Hennepin County, MN',
    'New York County, NY',
    'Los Angeles County, CA'
]

colors = [
    'royalblue',
    'firebrick',
    'forestgreen'
]

f1 = figure(width = 900, height = 540, x_range=[1970, 2013], title='PCPI for counties in which the author has lived',\
           x_axis_label='Year', y_axis_label='PCPI (2015 Dollars, Thousands)')

f2 = figure(width = 900, height = 540, x_range=f1.x_range, title='Population of counties in which the author has lived',\
           x_axis_label='Year', y_axis_label='Log County Population')

x = PCPI_norm.index.year

for color, name in zip(colors, names):
    FIPS = counties_inv[name]
    f1.line(x, PCPI_norm[FIPS+'_PCPI'], color=color, legend=name)
    f2.line(x, POP_log[FIPS+'_POP'], color=color, legend=name)

f = gridplot([[f1], [f2]])
show(f)

### Let's take a look at all counties in a single state:

In [ ]:
state = 'MN'
counties_in_state = [name for name in counties_inv.keys() if name.split(',')[1].strip() == state]

f = figure(width = 900, height = 540, x_range=[1970, 2013], y_range=[0, 100], title='PCPI in '+state, \
           x_axis_label='Year', y_axis_label='PCPI (2015 Dollars, Thousands)')

for c in counties_in_state:
    try:
        column_name = counties_inv[c]+'_PCPI'
        f.line(PCPI_norm.index.year, PCPI_norm[column_name].values)
    except KeyError:
        pass
    
show(f)

### Looks cool, but it's a little unwieldy. Let's compare just the 10 most populous counties and the national average:

In [ ]:
# first, let's get a list of FIPS codes with valid data so that we dont raise KeyErrors
valid_FIPS = [col[:5] for col in POP_data.columns]

In [ ]:
n = 10 # if using brewer below, n must be between 2 and 10

state = 'MN'
counties_in_state = [name for name in counties_inv.keys() if name.split(',')[1].strip() == state]

# Let's make a dataframe from the POP_log dataframe which only has the state of interest
# and then take the n highest populated counties at 2013
FIPS_in_state = [FIPS for FIPS, name in counties.items() if name.split(',')[1].strip() == state]
top = POP_log[[FIPS+'_POP' for FIPS in FIPS_in_state if FIPS in valid_FIPS]]['2013'].max().sort_values(ascending=False).index[:n]

# Let's make another new dataframe, this time for PCPI
# We want the top n counties and the national average
top_PCPI = PCPI_norm[[top_i[:5]+'_PCPI' for top_i in top]]
top_PCPI.columns = [counties[colname[:5]] for colname in top_PCPI.columns] # renaming is easier before we add natl_avg
natl_avg = PCPI_norm.mean(axis = 1)
natl_avg.name = "National Average"
top_PCPI = top_PCPI.join(natl_avg)

# We'll sort the data by the maximum PCPI, so data will be ordered
# in legend and coloring by highest to lowest
cols_in_order = top_PCPI.max().sort_values(ascending=False).index
x  = top_PCPI.index.year
colors = brewer['Spectral'][n+1][::-1]

fig_title = 'PCPI of {} most populous counties in {}'.format(n, state)
f = figure(width = 900, height = 540, x_range=[1970, 2013], y_range=[0, 100], title=fig_title, \
           x_axis_label='Year', y_axis_label='PCPI (2015 Dollars, Thousands)')

for color, column in zip(colors, cols_in_order):
    f.circle(x, top_PCPI[column].values, fill_color=color, line_color='black', size=8, legend=column)

    
f.legend.orientation = 'top_left'


show(f)

### To better assess comparative economic growth, lets normalize the PCPI of each county (and the national average) to the value for each at 1970:

In [ ]:
n = 10 # if using brewer below, n must be between 2 and 10

state = 'MN'
counties_in_state = [name for name in counties_inv.keys() if name.split(',')[1].strip() == state]

# Let's make a dataframe from the POP_log dataframe which only has the state of interest
# and then take the n highest populated counties at 2013
FIPS_in_state = [FIPS for FIPS, name in counties.items() if name.split(',')[1].strip() == state]
top = POP_log[[FIPS+'_POP' for FIPS in FIPS_in_state if FIPS in valid_FIPS]]['2013'].max().sort_values(ascending=False).index[:n]

# Let's make another new dataframe, this time for PCPI
# We want the top n counties and the national average
top_PCPI = PCPI_norm[[top_i[:5]+'_PCPI' for top_i in top]]
top_PCPI.columns = [counties[colname[:5]] for colname in top_PCPI.columns] # renaming is easier before we add natl_avg
natl_avg = PCPI_norm.mean(axis = 1)
natl_avg.name = "National Average"
top_PCPI = top_PCPI.join(natl_avg)

# We'll sort the data by the maximum PCPI, so data will be ordered
# in legend and coloring by highest to lowest
cols_in_order = top_PCPI.max().sort_values(ascending=False).index
x  = top_PCPI.index.year
colors = brewer['Spectral'][n+1][::-1]

fig_title = 'Growth of mean income in {} most populous counties in {}'.format(n, state)
f = figure(width = 900, height = 540, x_range=[1970, 2013], y_range=[0, 2.5], title=fig_title, \
           x_axis_label='Year', y_axis_label='Normalized Income Growth since 1970')

for color, column in zip(colors, cols_in_order):
    y = top_PCPI[column].values / top_PCPI[column].values[0]
    f.circle(x, y, fill_color=color, line_color='black', size=8, legend=column)

    
f.legend.orientation = 'bottom_right'


show(f)

### Looks like Ramsey County (home of St. Paul) fared worse after the 2008 financial crisis than the other major counties in Minnesota.

### Let's return to the first plot of PCPI for counties in Minnesota:
What is going on with that huge spike at 1973? It doesn't happen for any of the 10 most populous counties.

In [ ]:
state = 'MN'
counties_in_state = [name for name in counties_inv.keys() if name.split(',')[1].strip() == state]

f = figure(width = 900, height = 540, x_range=[1971, 1975], y_range=[0, 60], title='PCPI in '+state, \
           x_axis_label='Year', y_axis_label='PCPI (2015 Dollars, Thousands)')

for c in counties_in_state:
    try:
        column_name = counties_inv[c]+'_PCPI'
        f.line(PCPI_norm.index.year, PCPI_norm[column_name].values)
    except KeyError:
        pass
    
show(f)

### Let's get a list of counties, sorted by their proportional growth at 1973, and see if there are some unifying properties...

In [ ]:
state = 'MN'
FIPS_in_state = [FIPS for FIPS, name in counties.items() if name.split(',')[1].strip() == state]

in_state = PCPI_norm[[FIPS+'_PCPI' for FIPS in FIPS_in_state]]
in_state.columns = [counties[colname[:5]] for colname in in_state.columns]
natl_avg = PCPI_norm.mean(axis = 1)
natl_avg.name = "National Average"
in_state = in_state.join(natl_avg)

growth = in_state.diff() / in_state

# convert single row from in_state to series
growth['1973'].squeeze().sort_values(ascending=False)

### It seems that those counties which had the greatest proportional growth in 1973 are also among the [highest producers of corn in the state](http://www.nass.usda.gov/Statistics_by_State/Minnesota/Publications/County_Estimates/2015/MN_CtyEst_Soybean_%2013-14.pdf). 1973 was also when [oil prices rose](https://en.wikipedia.org/wiki/1973_oil_crisis) to excessive levels. This sudden increase in income may reflect an increased demand for corn, which is used to produce ethanol that in turn is used as diluent for gasoline. As oil becomes more expensive, it makes sense that lower grades of gasoline would be in higher demand, and thus more ethanol would be needed to produce it.